In [4]:
import os, json, time, random, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

import optuna


# ============================================================
# CONFIG (너가 원한 기본 경로 유지)
# ============================================================
TRAIN_PATH = "../../data/raw/train.csv"
TEST_PATH  = "../../data/raw/test_x.csv"

SEED = 42
N_FOLDS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Optuna (CPU 기준 "너무 오래 안 걸리게" 추천 세팅)
N_TRIALS = 50                 # CPU면 30~60 추천
TIMEOUT_SEC = None            # 예: 60*60 (1시간) 같은 제한도 가능
STUDY_NAME = "MLP_CLAUDE18"
STORAGE = None                # "sqlite:///optuna_mlp_claude18.db" 로 하면 중단/재개 가능

# Train (CPU 기준 빠르게)
EPOCHS = 45
PATIENCE = 6
BATCH_SIZE_DEFAULT = 512

print(f"🖥️ Device: {DEVICE}")
print(f"📂 Train path: {TRAIN_PATH}")
print(f"📂 Test  path: {TEST_PATH}")


def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


# ============================================================
# Data Load
# ============================================================
train_raw = pd.read_csv(TRAIN_PATH)
test_raw  = pd.read_csv(TEST_PATH)
train_raw["voted_bin"] = (train_raw["voted"] == 2).astype(int)
print(f"Train: {train_raw.shape}, Test: {test_raw.shape}")


# ============================================================
# Preprocess (클로드/너가 쓰던 핵심 방식)
# ============================================================
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 무응답 (0 → NaN)
    for col in ["education", "married", "urban", "engnat"]:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan

    # TP 0 → NaN
    for col in [f"tp{i:02d}" for i in range(1, 11)]:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan

    return df


# ============================================================
# Feature Engineering: CLAUDE18 전용
# ============================================================
def build_claude18_base(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # age_ord
    age_map = {"10s": 1, "20s": 2, "30s": 3, "40s": 4, "50s": 5, "60s": 6, "+70s": 7}
    df["age_ord"] = df["age_group"].map(age_map)

    # interaction
    df["age_edu"] = df["age_ord"] * df["education"]
    df["is_teenager"] = (df["age_ord"] == 1).astype(float)
    df["teen_low_edu"] = df["is_teenager"] * (df["education"] <= 2).astype(float)

    # wr_sum
    wr_cols = [f"wr_{i:02d}" for i in range(1, 14)]
    df["wr_sum"] = df[wr_cols].sum(axis=1)

    # TE combos
    df["age_edu_cat"] = df["age_group"].astype(str) + "_" + df["education"].astype(str)
    df["age_edu_mar_cat"] = (
        df["age_group"].astype(str) + "_" + df["education"].astype(str) + "_" + df["married"].astype(str)
    )
    return df


# ============================================================
# Target Encoding (fold 내부에서만, leak 방지 + smoothing)
# ============================================================
def target_encode(train_df, val_df, test_df, col, target_col="voted_bin", smoothing=10):
    global_mean = train_df[target_col].mean()
    agg = train_df.groupby(col)[target_col].agg(["mean", "count"])
    te = (agg["count"] * agg["mean"] + smoothing * global_mean) / (agg["count"] + smoothing)
    te_map = te.to_dict()

    return (
        train_df[col].map(te_map).fillna(global_mean).values,
        val_df[col].map(te_map).fillna(global_mean).values,
        test_df[col].map(te_map).fillna(global_mean).values,
    )


def make_te_5(train_df, val_df, test_df):
    # CLAUDE18: TE 5개만
    out = {"train": {}, "val": {}, "test": {}}
    specs = [
        ("age_group", 10),
        ("race", 10),
        ("religion", 10),
        ("age_edu_cat", 5),
        ("age_edu_mar_cat", 3),
    ]
    for c, sm in specs:
        tr, va, ts = target_encode(train_df, val_df, test_df, c, "voted_bin", sm)
        out["train"][f"{c}_te"] = tr
        out["val"][f"{c}_te"]   = va
        out["test"][f"{c}_te"]  = ts
    return out


# ============================================================
# CLAUDE18 Feature List (18개 고정)
# ============================================================
BASE5 = ["age_ord", "education", "married", "urban", "engnat"]
INTER4 = ["age_edu", "is_teenager", "teen_low_edu", "wr_sum"]
TP4 = ["tp07", "tp06", "tp04", "tp03"]
TE5 = ["age_group_te", "race_te", "religion_te", "age_edu_cat_te", "age_edu_mar_cat_te"]

CLAUDE18_COLS = BASE5 + INTER4 + TP4 + TE5


# ============================================================
# Dataset
# ============================================================
class TabDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]
        return self.X[idx], self.y[idx]


# ============================================================
# MLP Model (Optuna)
# ============================================================
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(prev, h),
                nn.BatchNorm1d(h),
                nn.SiLU(),
                nn.Dropout(dropout),
            ]
            prev = h
        layers += [nn.Linear(prev, 1)]
        self.net = nn.Sequential(*layers)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.net(x)


# ============================================================
# Train / Predict
# ============================================================
def train_one_fold(model, tr_loader, va_loader, y_va, lr, weight_decay, device):
    model.to(device)

    # imbalance 대응(간단 유지)
    pos_ratio = float(np.mean(y_va))
    pos_weight = torch.tensor([(1 - pos_ratio) / (pos_ratio + 1e-6)], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode="max", factor=0.5, patience=2)

    best_auc, best_state, no_imp = -1, None, 0

    for epoch in range(EPOCHS):
        model.train()
        for x, y in tr_loader:
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            loss = criterion(model(x), y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()

        model.eval()
        preds = []
        with torch.no_grad():
            for x, _y in va_loader:
                x = x.to(device)
                preds.append(torch.sigmoid(model(x)).cpu().numpy())
        preds = np.concatenate(preds).ravel()
        auc = roc_auc_score(y_va, preds)
        sched.step(auc)

        if auc > best_auc + 1e-5:
            best_auc = auc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            no_imp = 0
        else:
            no_imp += 1

        if no_imp >= PATIENCE:
            break

    if best_state is not None:
        model.load_state_dict(best_state)
    return best_auc


def predict(model, loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for x in loader:
            if isinstance(x, (tuple, list)):
                x = x[0]
            x = x.to(device)
            preds.append(torch.sigmoid(model(x)).cpu().numpy())
    return np.concatenate(preds).ravel()


# ============================================================
# Fold Matrix Builder (CLAUDE18 fixed)
# ============================================================
def make_X_claude18(tr_df, va_df, ts_df):
    # TE 생성(5개) 후 붙임
    te = make_te_5(tr_df, va_df, ts_df)

    # 이름 맞추기: age_group_te 등
    tr_df = tr_df.copy()
    va_df = va_df.copy()
    ts_df = ts_df.copy()

    tr_df["age_group_te"] = te["train"]["age_group_te"]
    va_df["age_group_te"] = te["val"]["age_group_te"]
    ts_df["age_group_te"] = te["test"]["age_group_te"]

    tr_df["race_te"] = te["train"]["race_te"]
    va_df["race_te"] = te["val"]["race_te"]
    ts_df["race_te"] = te["test"]["race_te"]

    tr_df["religion_te"] = te["train"]["religion_te"]
    va_df["religion_te"] = te["val"]["religion_te"]
    ts_df["religion_te"] = te["test"]["religion_te"]

    tr_df["age_edu_cat_te"] = te["train"]["age_edu_cat_te"]
    va_df["age_edu_cat_te"] = te["val"]["age_edu_cat_te"]
    ts_df["age_edu_cat_te"] = te["test"]["age_edu_cat_te"]

    tr_df["age_edu_mar_cat_te"] = te["train"]["age_edu_mar_cat_te"]
    va_df["age_edu_mar_cat_te"] = te["val"]["age_edu_mar_cat_te"]
    ts_df["age_edu_mar_cat_te"] = te["test"]["age_edu_mar_cat_te"]

    X_tr = tr_df[CLAUDE18_COLS].copy()
    X_va = va_df[CLAUDE18_COLS].copy()
    X_ts = ts_df[CLAUDE18_COLS].copy()

    # NaN -> train median
    for c in CLAUDE18_COLS:
        med = X_tr[c].median()
        if pd.isna(med):
            med = 0.0
        X_tr[c] = X_tr[c].fillna(med)
        X_va[c] = X_va[c].fillna(med)
        X_ts[c] = X_ts[c].fillna(med)

    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_tr.values)
    X_va = scaler.transform(X_va.values)
    X_ts = scaler.transform(X_ts.values)

    return X_tr, X_va, X_ts


# ============================================================
# Optuna Objective (MLP only, CLAUDE18 fixed)
# ============================================================
def objective(trial: optuna.Trial, train_all: pd.DataFrame, test_all: pd.DataFrame):
    # model params
    n_layers = trial.suggest_int("n_layers", 2, 4)
    base = trial.suggest_categorical("hidden_base", [128, 192, 256, 320])
    shrink = trial.suggest_float("shrink", 0.55, 0.85)
    hidden_dims = []
    cur = float(base)
    for _ in range(n_layers):
        hidden_dims.append(int(cur))
        cur = max(32.0, cur * shrink)

    dropout = trial.suggest_float("dropout", 0.1, 0.45)
    lr = trial.suggest_float("lr", 2e-4, 2e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [256, 512])

    y = train_all["voted_bin"].values.astype(np.float32)
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    fold_aucs = []
    for fold, (tr_idx, va_idx) in enumerate(skf.split(train_all, y), 1):
        tr_df = train_all.iloc[tr_idx].copy().reset_index(drop=True)
        va_df = train_all.iloc[va_idx].copy().reset_index(drop=True)
        ts_df = test_all.copy()

        X_tr, X_va, _ = make_X_claude18(tr_df, va_df, ts_df)

        y_tr = tr_df["voted_bin"].values.astype(np.float32)
        y_va = va_df["voted_bin"].values.astype(np.float32)

        tr_ds = TabDataset(X_tr, y_tr)
        va_ds = TabDataset(X_va, y_va)
        tr_loader = DataLoader(tr_ds, batch_size=batch_size, shuffle=True, drop_last=True)
        va_loader = DataLoader(va_ds, batch_size=batch_size, shuffle=False)

        model = MLP(input_dim=X_tr.shape[1], hidden_dims=hidden_dims, dropout=dropout)
        auc = train_one_fold(model, tr_loader, va_loader, y_va, lr, weight_decay, DEVICE)

        fold_aucs.append(auc)
        trial.report(float(np.mean(fold_aucs)), step=fold)

        if trial.should_prune():
            raise optuna.TrialPruned()

    mean_auc = float(np.mean(fold_aucs))
    print(
        f"[Trial {trial.number:03d}] AUC={mean_auc:.5f} | feat=18 | "
        f"layers={n_layers} base={base} shrink={shrink:.2f} drop={dropout:.2f} "
        f"lr={lr:.1e} wd={weight_decay:.1e} bs={batch_size}"
    )
    return mean_auc


# ============================================================
# Train best and predict
# ============================================================
def train_and_predict_best(best_params, train_all, test_all):
    n_layers = int(best_params["n_layers"])
    base = int(best_params["hidden_base"])
    shrink = float(best_params["shrink"])
    hidden_dims = []
    cur = float(base)
    for _ in range(n_layers):
        hidden_dims.append(int(cur))
        cur = max(32.0, cur * shrink)

    dropout = float(best_params["dropout"])
    lr = float(best_params["lr"])
    weight_decay = float(best_params["weight_decay"])
    batch_size = int(best_params["batch_size"])

    y = train_all["voted_bin"].values.astype(np.float32)
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    oof = np.zeros(len(train_all), dtype=np.float32)
    test_pred = np.zeros(len(test_all), dtype=np.float32)

    fold_aucs = []
    for fold, (tr_idx, va_idx) in enumerate(skf.split(train_all, y), 1):
        tr_df = train_all.iloc[tr_idx].copy().reset_index(drop=True)
        va_df = train_all.iloc[va_idx].copy().reset_index(drop=True)
        ts_df = test_all.copy()

        X_tr, X_va, X_ts = make_X_claude18(tr_df, va_df, ts_df)

        y_tr = tr_df["voted_bin"].values.astype(np.float32)
        y_va = va_df["voted_bin"].values.astype(np.float32)

        tr_ds = TabDataset(X_tr, y_tr)
        va_ds = TabDataset(X_va, y_va)
        ts_ds = TabDataset(X_ts)

        tr_loader = DataLoader(tr_ds, batch_size=batch_size, shuffle=True, drop_last=True)
        va_loader = DataLoader(va_ds, batch_size=batch_size, shuffle=False)
        ts_loader = DataLoader(ts_ds, batch_size=batch_size, shuffle=False)

        model = MLP(input_dim=X_tr.shape[1], hidden_dims=hidden_dims, dropout=dropout)
        auc = train_one_fold(model, tr_loader, va_loader, y_va, lr, weight_decay, DEVICE)
        fold_aucs.append(auc)

        oof[va_idx] = predict(model, va_loader, DEVICE)
        test_pred += predict(model, ts_loader, DEVICE) / N_FOLDS

        print(f"[BEST][Fold {fold}] AUC={auc:.5f}")

    oof_auc = roc_auc_score(train_all["voted_bin"], oof)
    print(f"\n✅ BEST OOF AUC: {oof_auc:.5f} | fold mean: {np.mean(fold_aucs):.5f}")
    return oof_auc, test_pred, fold_aucs


def auto_name(prefix, oof_auc):
    t = time.strftime("%m%d_%H%M%S")
    return f"{prefix}_auc{oof_auc:.5f}_{t}"


def main():
    set_seed(SEED)

    # clean + base feats
    train_clean = clean_data(train_raw)
    test_clean  = clean_data(test_raw)

    train_fe = build_claude18_base(train_clean)
    test_fe  = build_claude18_base(test_clean)

    sampler = optuna.samplers.TPESampler(seed=SEED)
    pruner  = optuna.pruners.MedianPruner(n_warmup_steps=2)

    study = optuna.create_study(
        direction="maximize",
        sampler=sampler,
        pruner=pruner,
        study_name=STUDY_NAME,
        storage=STORAGE,
        load_if_exists=True,
    )

    print("\n" + "="*80)
    print(f"🚀 OPTUNA START | trials={N_TRIALS} | timeout={TIMEOUT_SEC} | mode=CLAUDE18 fixed")
    print("="*80)

    study.optimize(lambda tr: objective(tr, train_fe, test_fe), n_trials=N_TRIALS, timeout=TIMEOUT_SEC)

    print("\n" + "="*80)
    print("🏁 OPTUNA DONE")
    print("="*80)
    print("Best AUC:", study.best_value)
    print("Best params:\n", json.dumps(study.best_params, indent=2))

    best_auc, test_pred, fold_aucs = train_and_predict_best(study.best_params, train_fe, test_fe)

    base = auto_name("submission_20_mlp_optuna_claude18", best_auc)

    sub = pd.DataFrame({
        "index": test_raw["index"] if "index" in test_raw.columns else np.arange(len(test_raw)),
        "voted": test_pred
    })
    sub_path = f"{base}.csv"
    sub.to_csv(sub_path, index=False)

    with open(f"{base}_params.json", "w") as f:
        json.dump(
            {"best_auc": float(best_auc), "fold_aucs": [float(x) for x in fold_aucs], "params": study.best_params},
            f,
            indent=2
        )

    print(f"\n💾 Saved: {sub_path}")
    print(f"💾 Saved: {base}_params.json")
    print(f"   pred range: [{test_pred.min():.4f}, {test_pred.max():.4f}]")


if __name__ == "__main__":
    main()


🖥️ Device: cpu
📂 Train path: ../../data/raw/train.csv
📂 Test  path: ../../data/raw/test_x.csv


[I 2026-01-30 15:33:24,636] A new study created in memory with name: MLP_CLAUDE18


Train: (45532, 79), Test: (11383, 77)

🚀 OPTUNA START | trials=50 | timeout=None | mode=CLAUDE18 fixed


[I 2026-01-30 15:34:30,845] Trial 0 finished with value: 0.7654244758908741 and parameters: {'n_layers': 3, 'hidden_base': 128, 'shrink': 0.5967983561008608, 'dropout': 0.12032926425886982, 'lr': 0.0014696236810540894, 'weight_decay': 6.358358856676247e-05, 'batch_size': 256}. Best is trial 0 with value: 0.7654244758908741.


[Trial 000] AUC=0.76542 | feat=18 | layers=3 base=128 shrink=0.60 drop=0.12 lr=1.5e-03 wd=6.4e-05 bs=256


...

[Trial 047] AUC=0.76590 | feat=18 | layers=4 base=320 shrink=0.62 drop=0.14 lr=1.4e-03 wd=2.0e-05 bs=512


[I 2026-01-30 16:47:00,388] Trial 48 finished with value: 0.7655329187549553 and parameters: {'n_layers': 4, 'hidden_base': 128, 'shrink': 0.6094742148323288, 'dropout': 0.14658825724143607, 'lr': 0.0017259024198359614, 'weight_decay': 1.6285018364427252e-05, 'batch_size': 512}. Best is trial 5 with value: 0.766338248422073.


[Trial 048] AUC=0.76553 | feat=18 | layers=4 base=128 shrink=0.61 drop=0.15 lr=1.7e-03 wd=1.6e-05 bs=512


[I 2026-01-30 16:48:45,631] Trial 49 finished with value: 0.7656728536699509 and parameters: {'n_layers': 4, 'hidden_base': 320, 'shrink': 0.6333614299118054, 'dropout': 0.11492602157245047, 'lr': 0.0014352070583865007, 'weight_decay': 5.6865076908556216e-05, 'batch_size': 256}. Best is trial 5 with value: 0.766338248422073.


[Trial 049] AUC=0.76567 | feat=18 | layers=4 base=320 shrink=0.63 drop=0.11 lr=1.4e-03 wd=5.7e-05 bs=256

🏁 OPTUNA DONE
Best AUC: 0.766338248422073
Best params:
 {
  "n_layers": 4,
  "hidden_base": 320,
  "shrink": 0.6666031869068446,
  "dropout": 0.19497216112086357,
  "lr": 0.0013482409221405526,
  "weight_decay": 1.1756010900231857e-05,
  "batch_size": 512
}
[BEST][Fold 1] AUC=0.77928
[BEST][Fold 2] AUC=0.76841
[BEST][Fold 3] AUC=0.75959
[BEST][Fold 4] AUC=0.75798
[BEST][Fold 5] AUC=0.76424

✅ BEST OOF AUC: 0.76570 | fold mean: 0.76590

💾 Saved: submission_20_mlp_optuna_claude18_auc0.76570_0130_165000.csv
💾 Saved: submission_20_mlp_optuna_claude18_auc0.76570_0130_165000_params.json
   pred range: [0.1241, 0.9954]
